In [73]:
from PIL import Image, ImageDraw
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import math
import random

In [74]:
def sin(a):
    return math.sin(a)
def cos(a):
    return math.cos(a)
def tan(a):
    return math.tan(a)
pi=math.pi
sqrt2=math.sqrt(2)

def deg2rad(t):
    return t/180*pi
def rad2deg(t):
    return t/pi*180

def rotatePoint(p,c,t):
    '''
    rotate a point p=(px,py) about a point c=(cx,cy) by t radians counterclockwise
    '''
    px=p[0]; py=p[1]; px-=c[0]; py-=c[1]
    rx=px*cos(t)-py*sin(t)+c[0]
    ry=px*sin(t)+py*cos(t)+c[1]
    return (rx,ry)

def midpoint(a,b):
    return (a[0]/2+b[0]/2,a[1]/2+b[1]/2)

In [75]:
im=Image.open('bluegradient500x500.jpeg')
draw=ImageDraw.Draw(im)


# Octagon Geometry

Define by center, radius, angle (radians)

**center**= (cx, cy)

**radius**=r

#### When **angle** = 0:

- N point is (cx,cy+r)

- NW point is (cx-r\*sqrt2/2, cy+r\*sqrt2/2)

- W point is (cx-r,cy)

- SW point is (cx-r\*sqrt2/2, cy-r\*sqrt2/2)

- S point is (cx,cy-r)

- SE point is (cx+r\*sqrt2/2, cy-r\*sqrt2/2)

- E point is (cx+r,cy)

- NE point is (cx+r\*sqrt2, cy+r\*sqrt2)

In [76]:
def OctagonPoints(c, r, t):
    '''
    Function for finding vertices of an octagon given: center coordinates c=(cx,cy), radius (r), and angle of rotation (t)
    '''
    cx=c[0]; cy=c[1];
    points=[
        (cx,cy+r),
        (cx-r*sqrt2/2,cy+r*sqrt2/2),
        (cx-r,cy),
        (cx-r*sqrt2/2,cy-r*sqrt2/2),
        (cx,cy-r),
        (cx+r*sqrt2/2,cy-r*sqrt2/2),
        (cx+r,cy),
        (cx+r*sqrt2/2,cy+r*sqrt2/2),
    ]
    for i in range(8):
        points[i]=rotatePoint(points[i],c,t)
    return points

def DrawOctagons(c,r1,r2,t,w,draw):
    '''
    Function for drawing entire board
    '''
    o1=OctagonPoints(c,r1,t); draw.polygon(o1,outline='black',fill='white',width=w)
    o2=OctagonPoints(c,r2,t); draw.polygon(o2,outline='black',fill='white',width=w)
    for i in range(8):
        draw.line([o1[i],o2[i]],fill='black',width=w)

# Chevron Geometry

Defined by center (of square), side length (of square)

**center** = (cx, cy)

**side length** = s

- N point is (cx,cy+s)

- NE point is (cx+s/2, cy+s/2)

- SE point is (cx+s/2, cy-s/2)

- Pocket is (cx, cy)

- SW point is (cx-s/2, cy-s/2)

- NW point is (cx-s/2, cy+s/2)

In [77]:
def ChevronPoints(c,s,t):
    '''
    Function for drawing an individual chevron given: center coordinates c=(cx,cy), side length (s), and angle of rotation (t)
    '''
    cx=c[0]; cy=c[1];
    points=[
        (cx,cy+s),
        (cx+s/2,cy+s/2),
        (cx+s/2,cy-s/2),
        (cx,cy),
        (cx-s/2,cy-s/2),
        (cx-s/2,cy+s/2)
    ]
    for i in range(6):
        points[i]=rotatePoint(points[i],c,t)
    return points

def DrawChevron(c,s,t,draw):
    draw.polygon(ChevronPoints(c,s,t),fill='red')

In [78]:
class CHEV:
    def __init__(self,x1,y1,x2,y2):
        self.midpoint1=(x1,y1)
        self.midpoint2=(x2,y2)
    def __init__(self,m1,m2):
        self.midpoint1=m1
        self.midpoint2=m2
class IMG:
    def __init__(self,numValidChevrons,validChevrons, legal1):
        self.chevsInCenter=numValidChevrons
        self.validChevrons=validChevrons
        self.legal=legal1

# Generator

In [79]:
def generateNRandom(n,im,draw,rigged):
    counter=0
    '''
    Image dimensions are 500x500
    rigged is how far the chevron can possibly fall to the octagon center
    '''
    
    minOctagonCenterX=100; maxOctagonCenterX=400
    minOctagonCenterY=100; maxOctagonCenterY=400
    minOctagonLargeRadius=100; maxOctagonLargeRadius=300
    maxRadiusRatio=3; minRadiusRatio=1.3
    minOctagonOutlineWidth=2; maxOctagonOutlineWidth=8
    minChevronSize=20; maxChevronSize=30

    octagonCenterX=random.randrange(minOctagonCenterX,maxOctagonCenterX)
    octagonCenterY=random.randrange(minOctagonCenterY,maxOctagonCenterY)
    minChevronCenterX=octagonCenterX-rigged
    maxChevronCenterX=octagonCenterX+rigged
    minChevronCenterY=octagonCenterY-rigged
    maxChevronCenterY=octagonCenterY+rigged
    octagonLargeRadius=random.randrange(minOctagonLargeRadius,maxOctagonLargeRadius)
    octagonRadiusRatio=random.uniform(minRadiusRatio,maxRadiusRatio)
    octagonSmallRadius=octagonLargeRadius//octagonRadiusRatio
    octagonOutlineWidth=random.randrange(minOctagonOutlineWidth,maxOctagonOutlineWidth)
    octagonAngle=random.uniform(0,2*pi)
    DrawOctagons((octagonCenterX,octagonCenterY),octagonLargeRadius,octagonSmallRadius,octagonAngle,octagonOutlineWidth,draw)
    innerOcto=Polygon(OctagonPoints((octagonCenterX,octagonCenterY),octagonSmallRadius,octagonAngle))
    chevronSize=random.randrange(minChevronSize,maxChevronSize)
    
    chevPolys=[]
    chevsPoints=[]
    chevs=[]

    for i in range(n):
        chevronCenterX=random.randrange(minChevronCenterX,maxChevronCenterX)
        chevronCenterY=random.randrange(minChevronCenterY,maxChevronCenterY)
        chevronAngle=random.uniform(0,2*pi)
        DrawChevron((chevronCenterX,chevronCenterY),chevronSize,chevronAngle,draw)
        chevPoints=ChevronPoints((chevronCenterX,chevronCenterY),chevronSize,chevronAngle)
        thisChevron=CHEV(midpoint(chevPoints[1],chevPoints[2]),midpoint(chevPoints[4],chevPoints[5]))
        chevPolys.append(Polygon(chevPoints))
        chevsPoints.append(chevPoints)
        allinside=True
        for cpt in chevPoints:
            pt=Point(cpt[0],cpt[1])
            if innerOcto.contains(pt):
                continue
            else:
                allinside=False
                break
        if allinside:
            chevs.append(thisChevron)
            counter+=1
    allDistinct=True
    for i in range(n-1):
        for j in range(i,n):
            for cpt in chevsPoints[i]:
                if chevPolys[j].contains(Point(cpt[0],cpt[1])):
                    allDistinct=False
                    break
    

    return IMG(counter,chevs,allDistinct)

In [80]:
id2num={}

iii=0

INFO={}

while (iii<100):
    im=Image.open('bluegradient500x500.jpeg')
    draw=ImageDraw.Draw(im)
    this=generateNRandom(random.randrange(1,5),im,draw,75)
    id='0'*(5-len(str(iii)))+str(iii)
    if this.legal:
        iii+=1
        INFO[id]=this
        im.save("IMG"+id+".jpeg","JPEG")

In [83]:
for i in INFO:
    print(i+":    "+str(INFO[i].chevsInCenter))

00000:    0
00001:    1
00002:    1
00003:    0
00004:    3
00005:    0
00006:    0
00007:    1
00008:    1
00009:    2
00010:    2
00011:    1
00012:    2
00013:    1
00014:    0
00015:    1
00016:    0
00017:    0
00018:    0
00019:    0
00020:    1
00021:    2
00022:    1
00023:    4
00024:    1
00025:    3
00026:    0
00027:    1
00028:    2
00029:    1
00030:    1
00031:    0
00032:    1
00033:    2
00034:    1
00035:    0
00036:    1
00037:    1
00038:    2
00039:    1
00040:    0
00041:    1
00042:    1
00043:    3
00044:    2
00045:    2
00046:    0
00047:    1
00048:    2
00049:    1
00050:    2
00051:    2
00052:    1
00053:    3
00054:    0
00055:    1
00056:    0
00057:    1
00058:    4
00059:    4
00060:    3
00061:    2
00062:    0
00063:    0
00064:    0
00065:    1
00066:    2
00067:    4
00068:    1
00069:    2
00070:    1
00071:    0
00072:    2
00073:    0
00074:    0
00075:    0
00076:    1
00077:    0
00078:    1
00079:    0
00080:    2
00081:    0
00082:    0
0008